In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Topic: EX2 - Turbofan RUL Prediction
**Task**: Predict the remaining useful life (RUL) of turbofan engines based on given sensor data (time series data). It is a forcasting problem, where the goal is to predict the number of cycles an engine will last before it fails.
**Data**: Turbofan engine degradation simulation data (NASA) - [Link](https://data.nasa.gov/dataset/Turbofan-Engine-Degradation-Simulation-Data-Set/vrks-gjie). See also in the topic [introduction notebook](https://github.com/nina-prog/damage-propagation-modeling/blob/2fb8c1a1102a48d7abbf04e4031807790a913a99/notebooks/Turbofan%20remaining%20useful%20life%20Prediction.ipynb).

**Subtasks**:
1. Perform a deep **exploratory data analysis (EDA)** on the given data.
2. Implement a more efficient **sliding window method** for time series data analysis. -> 🎯 **Focus on this task**
3. Apply **traditional machine learning methods** (SOTA) to predict the remaining useful life. Includes data preparation, feature extraction, feature selection, model selection, and model parameter optimization.
4. Create **neural network models** to predict the remaining useful life. Includes different architectures like Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), or Attention Models. Note: You can search for SOTA research papers and reproduce current state-of-the-art models.


# Imports + Settings

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!pip install -q tsfresh
!pip install -q colorlog

In [5]:
!pip install bayesian-optimization -q

In [45]:
# third-party libraries
import pandas as pd
import numpy as np
import os

import time
from tqdm.notebook import tqdm


# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html; Examples on how to approach classic classifiers
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ridge_regression, LogisticRegression, Lasso, LinearRegression

from tsfresh.feature_extraction import feature_calculators, MinimalFCParameters, EfficientFCParameters

# previous
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization
import sklearn.model_selection
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
!pip install -q tpot

In [8]:
from tpot import TPOTRegressor

In [9]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/PSDA_cml')

In [10]:
# source code
from src.utils import load_data, load_config, train_val_split_by_group
from src.data_cleaning import clean_data, format_dtype
from src.rolling_window_creator import calculate_RUL, RollingWindowDatasetCreator


In [11]:
# settings
sns.set_style("whitegrid")
sns.set_palette("Set2")
sns.set(rc={"figure.dpi":100, 'savefig.dpi':200})
sns.set_context('notebook')

In [12]:
np.random.seed(42)

# Paths

In [17]:
PATH_TO_CONFIG = "drive/MyDrive/PSDA_cml/configs/config.yaml"

# Load Config + Data

In [ ]:
#Debug

for i in MinimalFCParameters():
  print(i)

sum_values
median
mean
length
standard_deviation
variance
root_mean_square
maximum
absolute_maximum
minimum


In [15]:
config = load_config(PATH_TO_CONFIG) # config is dict

In [18]:
%%time

train_data, test_data,test_rul_data = load_data(config_path=PATH_TO_CONFIG, dataset_num=1)
train_data_2, test_data_2,test_rul_data_2 = load_data(config_path=PATH_TO_CONFIG, dataset_num=2)
train_data_3, test_data_3,test_rul_data_3 = load_data(config_path=PATH_TO_CONFIG, dataset_num=3)
train_data_4, test_data_4,test_rul_data_4 = load_data(config_path=PATH_TO_CONFIG, dataset_num=4)

2024-06-01 10:02:19 [src.utils:60] [INFO] >>>> Loading data set 1...


INFO:src.utils:Loading data set 1...


2024-06-01 10:02:20 [src.utils:89] [INFO] >>>> Loaded raw data for dataset 1.


INFO:src.utils:Loaded raw data for dataset 1.


2024-06-01 10:02:20 [src.utils:90] [INFO] >>>> Train Data: (20631, 26)


...

2024-06-01 10:02:24 [src.utils:91] [INFO] >>>> Test Data: (41214, 26)


INFO:src.utils:Test Data: (41214, 26)


2024-06-01 10:02:24 [src.utils:92] [INFO] >>>> Test RUL Data: (248, 1)


INFO:src.utils:Test RUL Data: (248, 1)


CPU times: user 2.56 s, sys: 191 ms, total: 2.75 s
Wall time: 4.72 s


In [ ]:
print("DS 1:" ,train_data.shape, "DS 2:" ,train_data_2.shape, "DS 3:" ,train_data_3.shape, "DS 4:" ,train_data_4.shape)

In [ ]:
print("DS 1:" ,train_data.columns, "DS 2:" ,train_data_2.columns, "DS 3:" ,train_data_3.columns, "DS 4:" ,train_data_4.columns)

# 📍 << Task 2: Classic Machine learning >>

[TEMPLATE]

Findings:
* Interpretation of plots
* or other key take aways from previous code

In [ ]:
# [TEMPLATE] - save processed data (as pickle)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_pickle(f"{config['paths']['processed_data_dir']}ex2_topic_{timestamp}.pkl")

In [ ]:
# [TEMPLATE] - save data predictions (as csv)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_csv(f"{config['paths']['prediction_dir']}ex2_topic_{timestamp}.csv", sep=',', decimal='.')

In [ ]:
# [TEMPLATE] - save plot results (as png)
fig = plt.figure(figsize=(9, 6))
timestamp = time.strftime("%Y%m%d-%H%M%S")
fig.savefig(f"{config['paths']['plot_dir']}ex2_topic_{timestamp}.png")

## Data Cleaning

Use the created in advanced functions to remove non-helpful data as determined by the EDA
(Currently without outlier removal)

In [ ]:
cleaned_train, cleaned_test = clean_data(train_data, test_data, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.5)

2024-05-31 19:33:26 [src.data_cleaning:134] [INFO] >>>> Cleaning train and test data...


INFO:src.data_cleaning:Cleaning train and test data...


2024-05-31 19:33:27 [src.data_cleaning:136] [INFO] >>>> Formatting column types...


INFO:src.data_cleaning:Formatting column types...


2024-05-31 19:33:27 [src.data_cleaning:69] [DEBUG] >>>> Found 0 categorical columns: []


...

INFO:src.data_cleaning:Data cleaning completed.


2024-05-31 19:33:27 [src.data_cleaning:173] [INFO] >>>> Original train DataFrame shape: (20631, 14), Resulting train DataFrame shape: (20631, 14)


INFO:src.data_cleaning:Original train DataFrame shape: (20631, 14), Resulting train DataFrame shape: (20631, 14)


2024-05-31 19:33:27 [src.data_cleaning:174] [INFO] >>>> Original test DataFrame shape: (13096, 14), Resulting test DataFrame shape: (13096, 14)


INFO:src.data_cleaning:Original test DataFrame shape: (13096, 14), Resulting test DataFrame shape: (13096, 14)


In [ ]:
train_data = format_dtype(train_data)

2024-05-31 19:33:31 [src.data_cleaning:69] [DEBUG] >>>> Found 0 categorical columns: []


DEBUG:src.data_cleaning:Found 0 categorical columns: []


In [ ]:
cleaned_train, cleaned_test = clean_data(train_data, test_data, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.5)
cleaned_train_2, cleaned_test_2 = clean_data(train_data_2, test_data_2, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.01)
cleaned_train_3, cleaned_test_3 = clean_data(train_data_3, test_data_3, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.3)
cleaned_train_4, cleaned_test_4 = clean_data(train_data_4, test_data_4, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.01)

In [ ]:
print("DS 1:" ,train_data.shape, "DS 2:" ,train_data_2.shape, "DS 3:" ,train_data_3.shape, "DS 4:" ,train_data_4.shape)
print( )
print("DS 1:" ,train_data.columns,"\n", "DS 2:" ,train_data_2.columns,"\n", "DS 3:" ,train_data_3.columns,"\n", "DS 4:" ,train_data_4.columns)

## Feature Engineering

Choose a set of feature options for tsfresh windowing


In [20]:
# Currently using minimal to ease optimization so feature_list is not necessary
feature_list_ds_1 = ["c3", "quantile", "mean", "root_mean_square", "median", "time_reversal_asymmetry_statistic", "absolute_maximum", "maximum", "minimum", "agg_autocorrelation", "autocorrelation" ]
# feature_list for dataset 1




#currentpath = os.getcwd()
#ft_list = pd.read_pickle(currentpath+ "/data/processed/dataset1_remaining_features_0521.pkl")

In [19]:
min_ts = 29
max_ts = 30

## Windowing

via tsfresh

In [21]:
rwCreator_std = RollingWindowDatasetCreator(max_timeshift=max_ts,min_timeshift=min_ts,feature_extraction_mode= 'minimal',feature_list=["median"])
rwCreator_fte = RollingWindowDatasetCreator(max_timeshift=max_ts,min_timeshift=min_ts,feature_extraction_mode= 'custom',feature_list=feature_list)

NameError: name 'feature_list' is not defined

In [ ]:
X_train_std, y_train_std, X_test_std, y_test_std = rwCreator_std.create_rolling_windows_datasets(train_data=cleaned_train, test_data=cleaned_test,test_RUL_data=test_rul_data,)
X_train_fte, y_train_fte, X_test_fte, y_test_fte = rwCreator_fte.create_rolling_windows_datasets(train_data=cleaned_train, test_data=cleaned_test,test_RUL_data=test_rul_data,)

2024-05-31 19:37:09 [src.rolling_window_creator:128] [INFO] >>>> Creating rolling windows for train data...


INFO:src.rolling_window_creator:Creating rolling windows for train data...
Rolling: 100%|██████████| 362/362 [00:27<00:00, 13.27it/s] 


2024-05-31 19:37:37 [src.rolling_window_creator:134] [INFO] >>>> Extracting features for train data...


INFO:src.rolling_window_creator:Extracting features for train data...
Feature Extraction: 100%|██████████| 212772/212772 [02:10<00:00, 1627.23it/s]


2024-05-31 19:39:56 [src.rolling_window_creator:142] [INFO] >>>> Calculating target for train data...


...

INFO:src.rolling_window_creator:Shape of y_train: (17731, 1)


2024-05-31 19:54:44 [src.rolling_window_creator:173] [INFO] >>>> Shape of X_test: (100, 396)


INFO:src.rolling_window_creator:Shape of X_test: (100, 396)


2024-05-31 19:54:44 [src.rolling_window_creator:174] [INFO] >>>> Shape of y_test: (100, 1)


INFO:src.rolling_window_creator:Shape of y_test: (100, 1)


In [ ]:
X_train_2, y_train_2, X_test_2, y_test_2 = rwCreator.create_rolling_windows_datasets(train_data=cleaned_train_2, test_data=cleaned_test_2,test_RUL_data=test_rul_data_2,)
X_train_3, y_train_3, X_test_3, y_test_3 = rwCreator.create_rolling_windows_datasets(train_data=cleaned_train_3, test_data=cleaned_test_3,test_RUL_data=test_rul_data_3,)
X_train_4, y_train_4, X_test_4, y_test_4 = rwCreator.create_rolling_windows_datasets(train_data=cleaned_train_4, test_data=cleaned_test_4,test_RUL_data=test_rul_data_4,)

## Scaling

- StandardScaler

In [ ]:
scaler_1 = StandardScaler()
scaler_2 = StandardScaler()

In [ ]:
X_train_std = scaler_1.fit_transform(X_train_std)
X_test_std = scaler_1.fit_transform(X_test_std)

X_train_fte = scaler_2.fit_transform(X_train_fte)
X_test_fte = scaler_2.fit_transform(X_test_fte)

In [ ]:
# Nachdem Hinweis am Montag in der Besprechung hinzugefügt. Verbesserung von ~10 pts. im RSME gesehen.
y_train_std = y_train_std.clip(upper=125)
y_train_fte = y_train_fte.clip(upper=125)

## Feature Selection

- RFE
- SimpleCorrApproach

## Models
- RandomForestClassifier - niklas
- SVC
- MLP - niklas
- GPC
- KNeighboursClassifier - niklas
- Gaussian Naive Bayes
- AdaBoostClassifier - niklas
- QuadraticDiscrimantAnalysis


In [ ]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(gamma=2, C=1, random_state=42),
    GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
    RandomForestClassifier(
        max_depth=5, n_estimators=10, max_features=1, random_state=42
    ),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    AdaBoostClassifier(algorithm="SAMME", random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),

]

In [ ]:
regressors = [
    KNeighborsRegressor(3),
    SVR(kernel="linear", C=0.025),
    SVR(gamma=2, C=1),
    GaussianProcessRegressor(1.0 * RBF(1.0), random_state=42),
    #RandomForestRegressor(),#max_depth=5, n_estimators=10, max_features=5, random_state=42
    MLPRegressor(alpha=1, max_iter=1000, random_state=42),
    AdaBoostRegressor(random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

## Training & Evaluation

In [ ]:
type(y_train.values.ravel())

In [ ]:
#for clf in regressors:
#
#    clf.fit(X_train, y_train.values.ravel())
#    y_pred = clf.predict(X_test)
#
#    print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

    #rmse = sklearn.model_selection.cross_val_score(clf, X=X_train, y=y_train, cv=5, scoring='root_mean_squared_error')
    #print(rmse)


In [ ]:
%%time
# TEST between minimal VS Feat Eng

rgr_std  = KNeighborsRegressor(3)
rgr_std.fit(X_train_std, y_train_std.values.ravel())
y_pred_std = rgr_std.predict(X_test_std)
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test_std, y_pred_std)))


rgr_fte  = KNeighborsRegressor(3)
rgr_fte.fit(X_train_fte, y_train_fte.values.ravel())
y_pred_fte = rgr_fte.predict(X_test_fte)
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test_fte, y_pred_fte)))


25.071608555406964
22.376475742767596
CPU times: user 143 ms, sys: 20 ms, total: 163 ms
Wall time: 202 ms


In [ ]:
%%time
rgr  = SVR(kernel="linear", C=0.025)

rgr.fit(X_train, y_train.values.ravel())
y_pred = rgr.predict(X_test)

print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

24.416745560533734
CPU times: user 59.3 s, sys: 314 ms, total: 59.6 s
Wall time: 1min 6s


In [ ]:
%%time
rgr  = SVR(gamma=2, C=1)

rgr.fit(X_train, y_train.values.ravel())
y_pred = rgr.predict(X_test)

print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

48.81927793822187
CPU times: user 1min 11s, sys: 245 ms, total: 1min 11s
Wall time: 1min 12s


In [ ]:
#rgr  = GaussianProcessRegressor(1.0 * RBF(1.0), random_state=42)

rgr.fit(X_train, y_train.values.ravel())
y_pred = rgr.predict(X_test)

print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
%%time
# TEST between minimal VS Feat Eng

rgr_std  = RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1, random_state=42)
rgr_std.fit(X_train_std, y_train_std.values.ravel())
y_pred_std = rgr_std.predict(X_test_std)
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test_std, y_pred_std)))

rgr_fte  = RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1, random_state=42)
rgr_fte.fit(X_train_fte, y_train_fte.values.ravel())
y_pred_fte = rgr_fte.predict(X_test_fte)
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test_fte, y_pred_fte)))

24.180103379729868
23.656090593432143
CPU times: user 197 ms, sys: 15 ms, total: 212 ms
Wall time: 229 ms


In [ ]:
%%time
rgr  = MLPRegressor(alpha=1, max_iter=1000, random_state=42)
rgr.fit(X_train, y_train.values.ravel())
y_pred = rgr.predict(X_test)

print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

30.842518615046394
CPU times: user 3min 48s, sys: 2min 22s, total: 6min 11s
Wall time: 4min 4s


In [ ]:
%%time
rgr  = AdaBoostRegressor(random_state=42)
rgr.fit(X_train, y_train.values.ravel())
y_pred = rgr.predict(X_test)

print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

25.612671397542982
CPU times: user 29.1 s, sys: 41.3 ms, total: 29.1 s
Wall time: 30.6 s


In [ ]:
%%time
rgr  = GaussianNB()

rgr.fit(X_train, y_train.values.ravel())
y_pred = rgr.predict(X_test)

print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

60.878814048895535
CPU times: user 89.9 ms, sys: 45.9 ms, total: 136 ms
Wall time: 85.8 ms


In [ ]:
rgr  = QuadraticDiscriminantAnalysis()

rgr.fit(X_train, y_train)
y_pred = rgr.predict(X_test)

print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
rgr  = KernelRidge()

rgr.fit(X_train, y_train.values.ravel())
y_pred = rgr.predict(X_test)

print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
%%time
# TEST between minimal VS Feat Eng

rgr_std  = Lasso()
rgr_std.fit(X_train_std, y_train_std.values.ravel())
y_pred_std = rgr_std.predict(X_test_std)
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test_std, y_pred_std)))

rgr_fte  = Lasso()
rgr_fte.fit(X_train_fte, y_train_fte.values.ravel())
y_pred_fte = rgr_fte.predict(X_test_fte)
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test_fte, y_pred_fte)))


22.454939897580797
19.546604449298922
CPU times: user 6.4 s, sys: 1.47 s, total: 7.87 s
Wall time: 4.11 s


In [ ]:
%%time
rgr  = LinearRegression()

rgr.fit(X_train, y_train.values.ravel())
y_pred = rgr.predict(X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))

223930.8766704401
CPU times: user 346 ms, sys: 48.8 ms, total: 394 ms
Wall time: 251 ms


In [ ]:
%%time
rgr  = LogisticRegression()

rgr.fit(X_train, y_train.values.ravel())
y_pred = rgr.predict(X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)))

37.078834933152905
CPU times: user 20.4 s, sys: 13.1 s, total: 33.5 s
Wall time: 20.2 s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
%%time
# TEST between minimal VS Feat Eng

rgr_std  = GradientBoostingRegressor()
rgr_std.fit(X_train_std, y_train_std.values.ravel())
y_pred_std = rgr_std.predict(X_test_std)
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test_std, y_pred_std)))

rgr_fte  = GradientBoostingRegressor()
rgr_fte.fit(X_train_fte, y_train_fte.values.ravel())
y_pred_fte = rgr_fte.predict(X_test_fte)
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test_fte, y_pred_fte)))



22.121312383899582
17.64114406542858
CPU times: user 5min 40s, sys: 346 ms, total: 5min 40s
Wall time: 5min 43s


In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
%%time
# TEST between minimal VS Feat Eng

rgr_std = XGBRegressor(n_estimators=3, max_depth=1, learning_rate=0.211)
rgr_std.fit(X_train_std, y_train_std.values.ravel())
y_pred_std = rgr_std.predict(X_test_std)
print(np.sqrt(mean_squared_error(y_test_std, y_pred_std)))


rgr_fte = XGBRegressor(n_estimators=3, max_depth=1, learning_rate=0.211)
rgr_fte.fit(X_train_fte, y_train_fte.values.ravel())
y_pred_fte = rgr_fte.predict(X_test_fte)
print(np.sqrt(mean_squared_error(y_test_fte, y_pred_fte)))

31.723973334531024
31.826748538652865
CPU times: user 2.42 s, sys: 55.7 ms, total: 2.47 s
Wall time: 2.52 s


## First Observations

Size is too big -> notebook outsourced to colab


## Optimizations

-- TODO --

In [ ]:
def hyperparameter_function(clf):
    ''' Hyperparameter Opti
    '''
    classifier = clf
    rmse = sklearn.model_selection.cross_val_score(classifier, X=X_train, y=y_train, cv=5, scoring='root_mean_squared_error')
    return rmse



# Bounded region of parameter space
    pbounds = {'neighbours': (3, 7)}

    optimizer = BayesianOptimization(
        f=hyperparameter_function_knn,
        pbounds=pbounds,
        random_state=17,
        allow_duplicate_points= True
    )

    optimizer.maximize(
        init_points=50,
        n_iter=100,
    )

## Testing

- Operation Setting Dependency in Datasets 2 & 4
- Basic Loop with 3 Regressors

In [ ]:
train_data_2_wRUL = calculate_RUL(train_data_2, time_column='Cycle', group_column='UnitNumber')
train_data_4_wRUL = calculate_RUL(train_data_4, time_column='Cycle', group_column='UnitNumber')

In [ ]:
print(train_data_2.shape)
print(train_data_2_wRUL.shape)
print(train_data_2_wRUL.columns)

(53759, 26)
(53759, 27)
Index(['UnitNumber', 'Cycle', 'Operation Setting 1', 'Operation Setting 2',
       'Operation Setting 3', 'Sensor Measure 1', 'Sensor Measure 2',
       'Sensor Measure 3', 'Sensor Measure 4', 'Sensor Measure 5',
       'Sensor Measure 6', 'Sensor Measure 7', 'Sensor Measure 8',
       'Sensor Measure 9', 'Sensor Measure 10', 'Sensor Measure 11',
       'Sensor Measure 12', 'Sensor Measure 13', 'Sensor Measure 14',
       'Sensor Measure 15', 'Sensor Measure 16', 'Sensor Measure 17',
       'Sensor Measure 18', 'Sensor Measure 19', 'Sensor Measure 20',
       'Sensor Measure 21', 'RUL'],
      dtype='object')


In [ ]:
#print("Dataset1:", train_data['Operation Setting 1'].value_counts(), train_data['Operation Setting 2'].value_counts(), train_data['Operation Setting 3'].value_counts())
print("Dataset2:", train_data_2_wRUL['Operation Setting 1'].value_counts(), train_data_2_wRUL['Operation Setting 2'].value_counts(), train_data_2_wRUL['Operation Setting 3'].value_counts())
print("Dataset4:", train_data_4_wRUL['Operation Setting 1'].value_counts(), train_data_4_wRUL['Operation Setting 2'].value_counts(), train_data_4_wRUL['Operation Setting 3'].value_counts())

Dataset2: Operation Setting 1
0.0020     310
0.0004     308
0.0026     293
0.0029     285
0.0022     281
          ... 
24.9980     43
25.0080     41
10.0080     41
34.9980     39
19.9980     32
Name: count, Length: 536, dtype: int64 Operation Setting 2
0.8400    11106
0.0000     4187
0.2500     4138
0.6200     4128
0.7000     4097
          ...  
0.7005      168
0.7020      124
0.6220       98
0.0020       93
0.2520       93
Name: count, Length: 105, dtype: int64 Operation Setting 3
100.0    45757
60.0      8002
Name: count, dtype: int64
Dataset4: Operation Setting 1
0.0022     339
0.0015     333
0.0023     324
0.0019     323
0.0006     323
          ... 
35.0080     48
34.9980     40
24.9980     37
10.0080     36
25.0080     36
Name: count, Length: 536, dtype: int64 Operation Setting 2
0.8400    12511
0.0000     4776
0.2500     4686
0.6200     4683
0.7000     4634
          ...  
0.7006      186
0.7020      124
0.0020      115
0.6220      115
0.2520      102
Name: count, Length: 105,

In [ ]:
tr_df_2_os2_84 = train_data_2_wRUL[train_data_2_wRUL['Operation Setting 2'] == 0.8400]
tr_df_2_os3_60 = train_data_2_wRUL[train_data_2_wRUL['Operation Setting 3'] == 60.0]


tr_df_4_os2_84 = train_data_4_wRUL[train_data_4_wRUL['Operation Setting 2'] == 0.8400]
tr_df_4_os3_60 = train_data_4_wRUL[train_data_4_wRUL['Operation Setting 3'] == 60.0]

In [ ]:
print("Dataset 2")
print("Operation Setting 2")
for col in tr_df_2_os2_84.columns:
  if abs(tr_df_2_os2_84['RUL'].corr(tr_df_2_os2_84[col])) >= 0.5:
    print(col)

print("Operation Setting 3")
for col in tr_df_2_os3_60.columns:
  if abs(tr_df_2_os3_60['RUL'].corr(tr_df_2_os3_60[col])) >= 0.5:
    print(col)

print("Dataset 3")
print("Operation Setting 2")
for col in tr_df_4_os2_84.columns:
  if abs(tr_df_4_os2_84['RUL'].corr(tr_df_4_os2_84[col])) >= 0.5:
    print(col)

print("Operation Setting 3")
for col in tr_df_4_os3_60.columns:
  if abs(tr_df_4_os3_60['RUL'].corr(tr_df_4_os3_60[col])) >= 0.5:
    print(col)


Dataset 2
Operation Setting 2
Cycle
Sensor Measure 4
Sensor Measure 11
Sensor Measure 15
RUL
Operation Setting 3
Cycle
Sensor Measure 3
Sensor Measure 4
Sensor Measure 11
Sensor Measure 15
Sensor Measure 17
RUL
Dataset 3
Operation Setting 2
Cycle
Sensor Measure 4
Sensor Measure 11
RUL
Operation Setting 3
Cycle
Sensor Measure 3
Sensor Measure 4
Sensor Measure 9
Sensor Measure 11
Sensor Measure 17
RUL


# Findings:

This eyperiment shows that certain sensors measures are still useful in Dataset 2 & 4

Thus they should get a manual droplist instead of automatic datacleaning

DS2: 3,4,11,15,17
DS4: 3,4,9,11,17


Basic pipeline with 3 regressors to evaluate tsfresh features

In [ ]:
#test
feat = "test"
rgr1 = 24.1
rgr2 = 24.2
rgr3 = 24.3
rgr4 = 24.4

d = {'Feature': feat, 'Regressor Results': [f"KNR: {rgr1}",f"RFR: {rgr2}", f"Lasso: {rgr3}", f"XBGr: {rgr4}"]}
df = pd.DataFrame(data=d)
df.to_pickle(f"drive/MyDrive/PSDA_cml/data/processed/ds4_tsf-feat_eff_results.pkl")

In [ ]:
df = pd.read_pickle("drive/MyDrive/PSDA_cml/data/processed/ds4_tsf-feat_eff_results.pkl")
df.head()

In [ ]:
#train_data
#clean data
#choose 1 ft from tsfresh.feature_extraction importfeature_calculators, MinimalFCParameters, EfficientFCParameters
#scale
#fit 1-3
#eval
#export findings

In [22]:
from tsfresh.feature_extraction import feature_calculators, MinimalFCParameters, EfficientFCParameters

3,4,11,15,17 DS4: 3,4,9,11,17

In [ ]:
cleaned_train_1, cleaned_test_1 = clean_data(train_data_4, test_data_4, method=None, ignore_columns=['UnitNumber', 'Cycle','Sensor Measure 3','Sensor Measure 4','Sensor Measure 9','Sensor Measure 11','Sensor Measure 15','Sensor Measure 17',], threshold_missing=0.1, threshold_corr=0.1)

In [ ]:
min_timeshift = 17
max_timeshift = 18

In [ ]:
print(train_data.columns)
cleaned_train_1.columns

In [ ]:
rwCreator = RollingWindowDatasetCreator(max_timeshift=max_timeshift,min_timeshift=min_timeshift,feature_extraction_mode= 'custom',feature_list=["feat"])
X_train_1, y_train_1, X_test_1, y_test_1 = rwCreator.create_rolling_windows_datasets(train_data=cleaned_train_1, test_data=cleaned_test_1,test_RUL_data=test_rul_data,)

In [ ]:
i= 0
for feat in EfficientFCParameters():
  #Counter
  print(i)
  i = i +1
  # RollingWindow
  rwCreator = RollingWindowDatasetCreator(max_timeshift=max_timeshift,min_timeshift=min_timeshift,feature_extraction_mode= 'custom',feature_list=[feat])
  X_train_1, y_train_1, X_test_1, y_test_1 = rwCreator.create_rolling_windows_datasets(train_data=cleaned_train_1, test_data=cleaned_test_1,test_RUL_data=test_rul_data_4,)
  # KNeighborsRegressor
  knr = KNeighborsRegressor(3)
  knr.fit(X_train_1, y_train_1.values.ravel())
  rgr1 = np.sqrt(sklearn.metrics.mean_squared_error(y_test_1, knr.predict(X_test_1)))
  # RandomForestRegressor
  rfr  = RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1, random_state=42)
  rfr.fit(X_train_1, y_train_1.values.ravel())
  rgr2 = np.sqrt(sklearn.metrics.mean_squared_error(y_test_1, rfr.predict(X_test_1)))
  # Lasso
  lr = Lasso()
  lr.fit(X_train_1, y_train_1.values.ravel())
  rgr3 = np.sqrt(sklearn.metrics.mean_squared_error(y_test_1, lr.predict(X_test_1)))
  # XGBRegressor
  xgbr = XGBRegressor(n_estimators=3, max_depth=1, learning_rate=0.211) # objective='binary:logistic'
  xgbr.fit(X_train_1, y_train_1.values.ravel())
  rgr4 = np.sqrt(mean_squared_error(y_test_1, xgbr.predict(X_test_1)))
  # Prev Data import
  df_in = pd.read_pickle("drive/MyDrive/PSDA_cml/data/processed/ds4_tsf-feat_eff_results.pkl")
  df = pd.DataFrame(data={'Feature': feat, 'Regressor Results': [f"KNR: {rgr1}",f"RFR: {rgr2}", f"Lasso: {rgr3}", f"XBGr: {rgr4}"]})
  df_out = pd.concat([df_in, df])
  df_out.to_pickle(f"drive/MyDrive/PSDA_cml/data/processed/ds4_tsf-feat_eff_results.pkl")
  print({'Feature': feat, 'Regressor Results': [f"KNR: {rgr1}",f"RFR: {rgr2}", f"Lasso: {rgr3}", f"XBGr: {rgr4}"]})

In [ ]:
df = pd.read_pickle("drive/MyDrive/PSDA_cml/data/processed/ds1_tsf-feat_eff_results.pkl")

In [ ]:
df

In [ ]:
# After compiling the best features for all 4 datasets

# Bayesian Optimization over the 4 chosen ones
# KNR RFR Lasso XGBR

In [ ]:
# Dataset 1 TPOT Hyperparameter Optimization

In [ ]:
min_ts_tpot = 29
max_ts_tpot = 30
cleaned_train_tpot, cleaned_test_tpot = clean_data(train_data, test_data, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.5)
rwCreator = RollingWindowDatasetCreator(max_timeshift=max_ts_tpot,min_timeshift=min_ts_tpot,feature_extraction_mode= 'minimal',feature_list=["median"])
X_train_tpot, y_train_tpot, X_test_tpot, y_test_tpot = rwCreator.create_rolling_windows_datasets(train_data=cleaned_train_tpot, test_data=cleaned_test_tpot,test_RUL_data=test_rul_data,)
y_train_tpot = y_train_tpot.clip(upper=125)

2024-05-31 11:58:23 [src.data_cleaning:134] [INFO] >>>> Cleaning train and test data...


INFO:src.data_cleaning:Cleaning train and test data...


2024-05-31 11:58:23 [src.data_cleaning:136] [INFO] >>>> Formatting column types...


INFO:src.data_cleaning:Formatting column types...


2024-05-31 11:58:23 [src.data_cleaning:69] [DEBUG] >>>> Found 0 categorical columns: []


...

INFO:src.rolling_window_creator:Shape of y_train: (17731, 1)


2024-05-31 12:00:49 [src.rolling_window_creator:173] [INFO] >>>> Shape of X_test: (100, 120)


INFO:src.rolling_window_creator:Shape of X_test: (100, 120)


2024-05-31 12:00:49 [src.rolling_window_creator:174] [INFO] >>>> Shape of y_test: (100, 1)


INFO:src.rolling_window_creator:Shape of y_test: (100, 1)


In [ ]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, mean_squared_error#, neg_mean_squared_error

In [ ]:
tpot = TPOTRegressor(generations=5, population_size=4, verbosity=2,
                      scoring='neg_mean_squared_error')
tpot.fit(X_train_tpot, y_train_tpot.values.ravel())

print(tpot.score(X_test_tpot, y_test_tpot))

Optimization Progress:   0%|          | 0/24 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -1647.5982802897245

Generation 2 - Current best internal CV score: -1647.5982802897245

Generation 3 - Current best internal CV score: -1644.2913001924958

Generation 4 - Current best internal CV score: -1644.2913001924958

Generation 5 - Current best internal CV score: -1644.2913001924958

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.2, min_samples_leaf=14, min_samples_split=16, n_estimators=100)
-1616.0362900072512


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Findings
- TPOT mit default Parametern dauert aufwärts von 4 Stunden
- Mit kleineren Werten(Generations = 2) keine sinnvollen Ergebnisse mse von ~1790
- Letzter Test von TPOT (Generations = 5, Pop_size = 4):

## Bayesian Optimization

Datensatz 1

In [59]:
## Laden des ersten Datensatzes
train_data_1_opt, test_data_1_opt,test_rul_data_1_opt = load_data(config_path=PATH_TO_CONFIG, dataset_num=1)
cleaned_train_1_opt, cleaned_test_1_opt = clean_data(train_data_1_opt, test_data_1_opt, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.5)

# Train Val Split
cl_train_1_opt, cl_val_1_opt = train_val_split_by_group(df = cleaned_train_1_opt,group = "UnitNumber",test_size = 0.18,n_splits = 2,random_state = 7)

## RollingWindowParameter
min_ts_1_opt = 29
max_ts_1_opt = 30
feature_list_ds_1 = ["c3", "quantile", "mean", "root_mean_square", "median", "time_reversal_asymmetry_statistic", "absolute_maximum", "maximum", "minimum", "agg_autocorrelation", "autocorrelation" ]

rwCreator = RollingWindowDatasetCreator(max_timeshift=max_ts_1_opt,min_timeshift=min_ts_1_opt,feature_extraction_mode= 'custom',feature_list=feature_list_ds_1)
#rwCreator = RollingWindowDatasetCreator(max_timeshift=max_ts_1_opt,min_timeshift=min_ts_1_opt,feature_extraction_mode= 'minimal',feature_list=["median"])


X_train_1_opt, y_train_1_opt = rwCreator._process_data(cl_train_1_opt, 'train')
X_val_1_opt, y_val_1_opt = rwCreator._process_data(cl_val_1_opt, 'train')
X_test_1_opt, y_test_1_opt = rwCreator._process_data(cleaned_test_1_opt, 'test', test_rul_data_1_opt)

y_train_1_opt = y_train_1_opt.clip(upper=125)



scaler = StandardScaler()
X_train_1_opt[2:] = scaler.fit_transform(X_train_1_opt[2:])
X_val_1_opt[2:] = scaler.fit_transform(X_val_1_opt[2:])
X_test_1_opt[2:] = scaler.fit_transform(X_test_1_opt[2:])

2024-06-01 13:15:34 [src.utils:60] [INFO] >>>> Loading data set 1...


INFO:src.utils:Loading data set 1...


2024-06-01 13:15:34 [src.utils:89] [INFO] >>>> Loaded raw data for dataset 1.


INFO:src.utils:Loaded raw data for dataset 1.


2024-06-01 13:15:34 [src.utils:90] [INFO] >>>> Train Data: (20631, 26)


...

INFO:src.rolling_window_creator:Calculating target for train data...


2024-06-01 13:29:15 [src.rolling_window_creator:128] [INFO] >>>> Creating rolling windows for test data...


INFO:src.rolling_window_creator:Creating rolling windows for test data...
Rolling: 100%|██████████| 303/303 [00:16<00:00, 18.63it/s] 


2024-06-01 13:29:33 [src.rolling_window_creator:134] [INFO] >>>> Extracting features for test data...


INFO:src.rolling_window_creator:Extracting features for test data...
Feature Extraction: 100%|██████████| 1200/1200 [00:03<00:00, 332.81it/s]


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import root_mean_squared_error

In [ ]:
def hyperparameter_function_knn(neighbours):
      """ Function for hyperparameter optimization
      """
      neighbours = neighbours.round().astype(int)
      knn_regressor = KNeighborsRegressor(n_neighbors=neighbours)
      metric = cross_val_score(knn_regressor, X=X_train_1_opt, y=y_train_1_opt, cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'neighbours': (1, 750)}

optimizer = BayesianOptimization(
  f=hyperparameter_function_knn,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   | neighb... |
-------------------------------------
| 1         | -22.76    | 221.7     |
| 2         | -22.71    | 398.4     |
| 3         | -22.92    | 144.4     |
| 4         | -23.71    | 51.86     |
| 5         | -22.72    | 590.5     |
| 6         | -22.71    | 492.6     |
| 7         | -22.71    | 478.5     |
| 8         | -22.71    | 432.1     |
| 9         | -24.22    | 30.26     |
| 10        | -22.72    | 269.0     |
| 11        | -22.71    | 435.5     |
| 12        | -22.77    | 744.9     |
| 13        | -22.74    | 668.2     |
| 14        | -22.71    | 331.4     |
| 15        | -22.72    | 546.6     |


KeyboardInterrupt: 

In [60]:
# Neigbors = 383
knn_regressor = KNeighborsRegressor(n_neighbors=383)
knn_regressor.fit(X_train_1_opt,y_train_1_opt )
y_pred_1_opt = knn_regressor.predict(X_test_1_opt)
print(np.sqrt(mean_squared_error(y_test_1_opt, y_pred_1_opt)))

23.361902580572174


In [ ]:
def hyperparameter_function_rf(n_estimators, max_features, ):
      """ Function for hyperparameter optimization
      """
      n_estimators = n_estimators.round().astype(int)
      max_features = max_features.round().astype(int)

      rf_regressor = RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,random_state=17,n_jobs=-1)
      metric = cross_val_score(rf_regressor, X=X_train_1_opt, y=y_train_1_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'n_estimators': (20, 500),'max_features': (1,1) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_rf,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   | max_fe... | n_esti... |
-------------------------------------------------
| 1         | -24.24    | 1.0       | 274.7     |
| 2         | -24.66    | 1.0       | 52.59     |
| 3         | -24.22    | 1.0       | 335.0     |


KeyboardInterrupt: 

In [80]:
# n_estimators=296 , max_features=4 -> 22.93
rf_regressor = RandomForestRegressor(n_estimators=296,max_features=4,random_state=17)
rf_regressor.fit(X_train_1_opt,y_train_1_opt.values.ravel())
y_pred_1_opt = rf_regressor.predict(X_test_1_opt)
print(np.sqrt(mean_squared_error(y_test_1_opt, y_pred_1_opt)))

18.476619772652022


In [70]:
def hyperparameter_function_lasso(alpha, max_iter, ):
      """ Function for hyperparameter optimization
      """
      max_iter = max_iter.round().astype(int)

      lasso_regressor = Lasso(alpha=alpha,max_iter=max_iter,random_state=17)
      metric = cross_val_score(lasso_regressor, X=X_train_1_opt, y=y_train_1_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'alpha': (0.001, 1),'max_iter': (100,10000) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_lasso,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   |   alpha   | max_iter  |
-------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.798e+06, tolerance: 1.996e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.723e+06, tolerance: 1.986e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.697e+06, tolerance: 1.985e

| 1         | -2.277e+0 | 0.2954    | 5.353e+03 |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.831e+03, tolerance: 1.983e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.728e+06, tolerance: 1.996e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.658e+06, tolerance: 1.986e

| 2         | -2.26e+05 | 0.1923    | 772.2     |


...

| 6         | -2.246e+0 | 0.9457    | 694.4     |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.298e+06, tolerance: 1.996e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.222e+06, tolerance: 1.986e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.196e+06, tolerance: 1.985e

| 7         | -2.211e+0 | 0.8642    | 8.785e+03 |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.365e+06, tolerance: 1.996e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.296e+06, tolerance: 1.986e+03
  model = cd_fast.enet_coordinate_descent(


KeyboardInterrupt: 

In [76]:
# alpha=0.05119 , max_iter=6559 -> 22.34
lasso_regressor = Lasso(alpha=0.0319,max_iter=656,random_state=17)
lasso_regressor.fit(X_train_1_opt,y_train_1_opt )
y_pred_1_opt = lasso_regressor.predict(X_test_1_opt)
print(np.sqrt(mean_squared_error(y_test_1_opt, y_pred_1_opt)))

1620.9163222136515


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.802e+06, tolerance: 2.482e+03
  model = cd_fast.enet_coordinate_descent(


In [ ]:
def hyperparameter_function_xgboost(eta, gamma ,max_depth,reg_lambda,reg_alpha ):
      """ Function for hyperparameter optimization
      """
      max_depth = max_depth.round().astype(int)

      xgb_regressor = XGBRegressor(eta=eta,gamma=gamma,max_depth=max_depth,reg_lambda=reg_lambda, reg_alpha=reg_alpha)
      metric = cross_val_score(xgb_regressor, X=X_train_1_opt, y=y_train_1_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'eta': (0, 1),'gamma': (0,2),'max_depth':(1,10),'reg_lambda':(1,1),'reg_alpha':(0,0) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_xgboost,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   |    eta    |   gamma   | max_depth | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
| 1         | -23.24    | 0.2947    | 1.061     | 2.724     | 0.0       | 1.0       |
| 2         | -27.72    | 0.6563    | 1.275     | 6.18      | 0.0       | 1.0       |
| 3         | -30.91    | 0.9457    | 0.1201    | 8.776     | 0.0       | 1.0       |
| 4         | -27.17    | 0.6524    | 1.104     | 6.378     | 0.0       | 1.0       |
| 5         | -22.62    | 0.2977    | 1.123     | 4.564     | 0.0       | 1.0       |
| 6         | -23.22    | 0.1439    | 0.3018    | 1.497     | 0.0       | 1.0       |
| 7         | -22.86    | 0.1988    | 1.663     | 6.112     | 0.0       | 1.0       |
| 8         | -22.71    | 0.159     | 1.354     | 2.066     | 0.0       | 1.0       |
| 9         | -29.26    | 0.7973    | 0.136     | 9.647     | 0.0       | 1.0       |
| 10        | -24.58    | 0.7436    | 1.775     | 2.20

In [77]:
# eta=0.09569 , gamma=0.05334 , max_depth=3.567 ,lambda=1 , alpha=0 -> 22.62
xgb_regressor = XGBRegressor(eta=0.09569,gamma=0.05334,max_depth=4,reg_lambda=1, reg_alpha=0)#eta=0.3,gamma=0,max_depth=6,reg_lambda=1, reg_alpha=0
xgb_regressor.fit(X_train_1_opt,y_train_1_opt )
y_pred_1_opt = xgb_regressor.predict(X_test_1_opt)

print(np.sqrt(mean_squared_error(y_test_1_opt, y_pred_1_opt)))

19.02862176067683


In [64]:
## Laden des dritten Datensatzes
train_data_3_opt, test_data_3_opt,test_rul_data_3_opt = load_data(config_path=PATH_TO_CONFIG, dataset_num=3)
cleaned_train_3_opt, cleaned_test_3_opt = clean_data(train_data_3_opt, test_data_3_opt, method=None, ignore_columns=['UnitNumber', 'Cycle'], threshold_missing=0.1, threshold_corr=0.5)

# Train Val Split
cl_train_3_opt, cl_val_3_opt = train_val_split_by_group(df = cleaned_train_3_opt,group = "UnitNumber",test_size = 0.18,n_splits = 2,random_state = 7)

## RollingWindowParameter
min_ts_3_opt = 29
max_ts_3_opt = 30
feature_list_ds_3 = ["c3", "quantile", "mean", "root_mean_square", "median", "time_reversal_asymmetry_statistic", "absolute_maximum", "maximum", "minimum", "agg_autocorrelation", "autocorrelation" ]

rwCreator = RollingWindowDatasetCreator(max_timeshift=max_ts_3_opt,min_timeshift=min_ts_3_opt,feature_extraction_mode= 'custom',feature_list=feature_list_ds_3)
#rwCreator = RollingWindowDatasetCreator(max_timeshift=max_ts_3_opt,min_timeshift=min_ts_3_opt,feature_extraction_mode= 'minimal',feature_list=["median"])


X_train_3_opt, y_train_3_opt = rwCreator._process_data(cl_train_3_opt, 'train')
X_val_3_opt, y_val_3_opt = rwCreator._process_data(cl_val_3_opt, 'train')
X_test_3_opt, y_test_3_opt = rwCreator._process_data(cleaned_test_3_opt, 'test', test_rul_data_3_opt)

y_train_3_opt = y_train_3_opt.clip(upper=125)

scaler = StandardScaler()
X_train_3_opt[2:] = scaler.fit_transform(X_train_3_opt[2:])
X_val_3_opt[2:] = scaler.fit_transform(X_val_3_opt[2:])
X_test_3_opt[2:] = scaler.fit_transform(X_test_3_opt[2:])

2024-06-01 13:31:40 [src.utils:60] [INFO] >>>> Loading data set 3...


INFO:src.utils:Loading data set 3...


2024-06-01 13:31:40 [src.utils:89] [INFO] >>>> Loaded raw data for dataset 3.


INFO:src.utils:Loaded raw data for dataset 3.


2024-06-01 13:31:40 [src.utils:90] [INFO] >>>> Train Data: (24720, 26)


...

INFO:src.rolling_window_creator:Calculating target for train data...


2024-06-01 13:42:56 [src.rolling_window_creator:128] [INFO] >>>> Creating rolling windows for test data...


INFO:src.rolling_window_creator:Creating rolling windows for test data...
Rolling: 100%|██████████| 475/475 [00:20<00:00, 22.83it/s] 


2024-06-01 13:43:17 [src.rolling_window_creator:134] [INFO] >>>> Extracting features for test data...


INFO:src.rolling_window_creator:Extracting features for test data...
Feature Extraction: 100%|██████████| 800/800 [00:02<00:00, 325.21it/s]


In [ ]:
def hyperparameter_function_knn(neighbours):
      """ Function for hyperparameter optimization
      """
      neighbours = neighbours.round().astype(int)
      knn_regressor = KNeighborsRegressor(n_neighbors=neighbours)
      metric = cross_val_score(knn_regressor, X=X_train_3_opt, y=y_train_3_opt, cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'neighbours': (1, 750)}

optimizer = BayesianOptimization(
  f=hyperparameter_function_knn,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   | neighb... |
-------------------------------------


KeyboardInterrupt: 

In [65]:
# Neigbors = 300
knn_regressor = KNeighborsRegressor(n_neighbors=300)
knn_regressor.fit(X_train_3_opt,y_train_3_opt )
y_pred_3_opt = knn_regressor.predict(X_test_3_opt)
print(np.sqrt(mean_squared_error(y_test_3_opt, y_pred_3_opt)))

24.766279241025376


In [ ]:
def hyperparameter_function_rf(n_estimators, max_features, ):
      """ Function for hyperparameter optimization
      """
      n_estimators = n_estimators.round().astype(int)
      max_features = max_features.round().astype(int)

      rf_regressor = RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,random_state=17,n_jobs=-1)
      metric = cross_val_score(rf_regressor, X=X_train_3_opt, y=y_train_3_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'n_estimators': (20, 500),'max_features': (1,1) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_rf,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   | max_fe... | n_esti... |
-------------------------------------------------
| 1         | -21.99    | 1.0       | 274.7     |
| 2         | -22.29    | 1.0       | 52.59     |
| 3         | -21.98    | 1.0       | 335.0     |
| 4         | -22.0     | 1.0       | 296.3     |
| 5         | -21.99    | 1.0       | 191.8     |
| 6         | -22.23    | 1.0       | 48.82     |
| 7         | -22.01    | 1.0       | 441.1     |
| 8         | -21.97    | 1.0       | 333.2     |
| 9         | -21.99    | 1.0       | 306.8     |
| 10        | -22.06    | 1.0       | 155.8     |
| 11        | -22.01    | 1.0       | 223.5     |
| 12        | -22.01    | 1.0       | 406.6     |
| 13        | -21.99    | 1.0       | 475.4     |
| 14        | -21.98    | 1.0       | 500.0     |
| 15        | -22.01    | 1.0       | 374.7     |
| 16        | -21.98    | 1.0       | 249.8     |
| 17        | -22.08    | 1.0       | 114.8     |
| 18        | -22.62    | 1.0       | 20.0      |


KeyboardInterrupt: 

In [69]:
# n_estimators= 333 , max_features= 1->
rf_regressor = RandomForestRegressor(n_estimators=333,max_features=1,random_state=17)#296,4
rf_regressor.fit(X_train_3_opt,y_train_3_opt.values.ravel())
y_pred_3_opt = rf_regressor.predict(X_test_3_opt)
print(np.sqrt(mean_squared_error(y_test_3_opt, y_pred_3_opt)))

23.361107460740012


In [ ]:
def hyperparameter_function_lasso(alpha, max_iter, ):
      """ Function for hyperparameter optimization
      """
      max_iter = max_iter.round().astype(int)

      lasso_regressor = Lasso(alpha=alpha,max_iter=max_iter,random_state=17)
      metric = cross_val_score(lasso_regressor, X=X_train_3_opt, y=y_train_3_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'alpha': (0.001, 1),'max_iter': (100,10000) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_lasso,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   |   alpha   | max_iter  |
-------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.874e+06, tolerance: 2.459e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.803e+06, tolerance: 2.460e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.686e+06, tolerance: 2.413e

| 1         | -4.436e+0 | 0.2954    | 5.353e+03 |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.811e+06, tolerance: 2.459e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.735e+06, tolerance: 2.460e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.622e+06, tolerance: 2.413e

| 2         | -4.393e+0 | 0.1923    | 772.2     |


...

| 12        | -1.619e+1 | 0.001     | 837.5     |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.166e+06, tolerance: 2.459e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.101e+06, tolerance: 2.460e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.974e+06, tolerance: 2.413e

| 13        | -4.765e+0 | 0.5926    | 5.937e+03 |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.141e+06, tolerance: 2.459e+03
  model = cd_fast.enet_coordinate_descent(


KeyboardInterrupt: 

In [78]:
# alpha= 0.1923 , max_iter= 772 ->
lasso_regressor = Lasso(alpha=0.1923,max_iter=772,random_state=17)
lasso_regressor.fit(X_train_3_opt,y_train_3_opt )
y_pred_3_opt = lasso_regressor.predict(X_test_3_opt)
print(np.sqrt(mean_squared_error(y_test_3_opt, y_pred_3_opt)))

8080.994375402995


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.159e+06, tolerance: 3.069e+03
  model = cd_fast.enet_coordinate_descent(


In [ ]:
def hyperparameter_function_xgboost(eta, gamma ,max_depth,reg_lambda,reg_alpha ):
      """ Function for hyperparameter optimization
      """
      max_depth = max_depth.round().astype(int)

      xgb_regressor = XGBRegressor(eta=eta,gamma=gamma,max_depth=max_depth,reg_lambda=reg_lambda, reg_alpha=reg_alpha)
      metric = cross_val_score(xgb_regressor, X=X_train_3_opt, y=y_train_3_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'eta': (0, 1),'gamma': (0,2),'max_depth':(1,10),'reg_lambda':(1,1),'reg_alpha':(0,0) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_xgboost,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   |    eta    |   gamma   | max_depth | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
| 1         | -18.06    | 0.2947    | 1.061     | 2.724     | 0.0       | 1.0       |
| 2         | -20.73    | 0.6563    | 1.275     | 6.18      | 0.0       | 1.0       |
| 3         | -23.58    | 0.9457    | 0.1201    | 8.776     | 0.0       | 1.0       |
| 4         | -20.65    | 0.6524    | 1.104     | 6.378     | 0.0       | 1.0       |
| 5         | -19.85    | 0.2977    | 1.123     | 4.564     | 0.0       | 1.0       |
| 6         | -17.62    | 0.1439    | 0.3018    | 1.497     | 0.0       | 1.0       |
| 7         | -19.65    | 0.1988    | 1.663     | 6.112     | 0.0       | 1.0       |
| 8         | -17.39    | 0.159     | 1.354     | 2.066     | 0.0       | 1.0       |
| 9         | -23.23    | 0.7973    | 0.136     | 9.647     | 0.0       | 1.0       |
| 10        | -19.24    | 0.7436    | 1.775     | 2.20

KeyboardInterrupt: 

In [68]:
# eta= 0.2079 , gamma= 1.342, max_depth= 2 ,lambda=1 , alpha=0 ->
xgb_regressor = XGBRegressor(eta=0.2079,gamma=1.342,max_depth=2,reg_lambda=1, reg_alpha=0)
xgb_regressor.fit(X_train_3_opt,y_train_3_opt )
y_pred_3_opt = xgb_regressor.predict(X_test_3_opt)

print(np.sqrt(mean_squared_error(y_test_3_opt, y_pred_3_opt)))

21.480055691867083


In [23]:
## Laden des zweiten Datensatzes
train_data_2_opt, test_data_2_opt,test_rul_data_2_opt = load_data(config_path=PATH_TO_CONFIG, dataset_num=2)
cleaned_train_2_opt, cleaned_test_2_opt = clean_data(train_data_2_opt, test_data_2_opt, method=None, ignore_columns=['UnitNumber', 'Cycle','Operation Setting 2','Operation Setting 3','Sensor Measure 2','Sensor Measure 3','Sensor Measure 4','Sensor Measure 8','Sensor Measure 9','Sensor Measure 11','Sensor Measure 15','Sensor Measure 17'], threshold_missing=0.1, threshold_corr=0.5)

# Train Val Split
cl_train_2_opt, cl_val_2_opt = train_val_split_by_group(df = cleaned_train_2_opt,group = "UnitNumber",test_size = 0.18,n_splits = 2,random_state = 7)

## RollingWindowParameter
min_ts_2_opt = 17
max_ts_2_opt = 18
feature_list_ds_2 = ["c3", "quantile", "mean", "median", "root_mean_square", "variance", "mean_abs_change", "standard_deviation", "skewness", "variation_coefficient", "last_location_of_maximum", "first_location_of_maximum"]

#rwCreator = RollingWindowDatasetCreator(max_timeshift=max_ts_2_opt,min_timeshift=min_ts_2_opt,feature_extraction_mode= 'custom',feature_list=feature_list_ds_3)
rwCreator = RollingWindowDatasetCreator(max_timeshift=max_ts_2_opt,min_timeshift=min_ts_2_opt,feature_extraction_mode= 'minimal',feature_list=["median"])


X_train_2_opt, y_train_2_opt = rwCreator._process_data(cl_train_2_opt, 'train')
X_val_2_opt, y_val_2_opt = rwCreator._process_data(cl_val_2_opt, 'train')
X_test_2_opt, y_test_2_opt = rwCreator._process_data(cleaned_test_2_opt, 'test', test_rul_data_2_opt)

y_train_2_opt = y_train_2_opt.clip(upper=125)

scaler = StandardScaler()
X_train_2_opt[2:] = scaler.fit_transform(X_train_2_opt[2:])
X_val_2_opt[2:] = scaler.fit_transform(X_val_2_opt[2:])
X_test_2_opt[2:] = scaler.fit_transform(X_test_2_opt[2:])

2024-06-01 10:03:10 [src.utils:60] [INFO] >>>> Loading data set 2...


INFO:src.utils:Loading data set 2...


2024-06-01 10:03:10 [src.utils:89] [INFO] >>>> Loaded raw data for dataset 2.


INFO:src.utils:Loaded raw data for dataset 2.


2024-06-01 10:03:10 [src.utils:90] [INFO] >>>> Train Data: (53759, 26)


...

INFO:src.rolling_window_creator:Calculating target for train data...


2024-06-01 10:09:07 [src.rolling_window_creator:128] [INFO] >>>> Creating rolling windows for test data...


INFO:src.rolling_window_creator:Creating rolling windows for test data...
Rolling: 100%|██████████| 367/367 [00:43<00:00,  8.35it/s]


2024-06-01 10:09:52 [src.rolling_window_creator:134] [INFO] >>>> Extracting features for test data...


INFO:src.rolling_window_creator:Extracting features for test data...
Feature Extraction: 100%|██████████| 2590/2590 [00:00<00:00, 2690.56it/s]


In [ ]:
def hyperparameter_function_knn(neighbours):
      """ Function for hyperparameter optimization
      """
      neighbours = neighbours.round().astype(int)
      knn_regressor = KNeighborsRegressor(n_neighbors=neighbours)
      metric = cross_val_score(knn_regressor, X=X_train_2_opt, y=y_train_2_opt, cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'neighbours': (1, 750)}

optimizer = BayesianOptimization(
  f=hyperparameter_function_knn,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   | neighb... |
-------------------------------------
| 1         | -29.66    | 221.7     |
| 2         | -31.11    | 398.4     |
| 3         | -28.77    | 144.4     |
| 4         | -27.48    | 51.86     |
| 5         | -32.11    | 590.5     |
| 6         | -31.68    | 492.6     |
| 7         | -31.61    | 478.5     |
| 8         | -31.33    | 432.1     |
| 9         | -27.16    | 30.26     |
| 10        | -30.12    | 269.0     |
| 11        | -34.51    | 1.0       |
| 12        | -27.18    | 31.27     |
| 13        | -27.76    | 71.06     |
| 14        | -28.13    | 97.85     |
| 15        | -28.43    | 120.3     |
| 16        | -29.24    | 179.2     |
| 17        | -32.64    | 750.0     |
| 18        | -30.6     | 327.3     |
| 19        | -32.4     | 669.0     |
| 20        | -31.9     | 540.0     |
| 21        | -30.86    | 361.7     |
| 22        | -30.36    | 297.2     |
| 23        | -29.45    | 199.9     |
| 24        | -32.26    | 629.0     |
| 25        

KeyboardInterrupt: 

In [29]:
# Neigbors = 20
knn_regressor = KNeighborsRegressor(n_neighbors=20)
knn_regressor.fit(X_train_2_opt,y_train_2_opt )
y_pred_2_opt = knn_regressor.predict(X_test_2_opt)
print(np.sqrt(mean_squared_error(y_test_2_opt, y_pred_2_opt)))
# default 54.41
# minimal 40.59 / 38
# custom+ 48.67
# custom  59.075

38.46334852215391


In [ ]:
def hyperparameter_function_rf(n_estimators, max_features, ):
      """ Function for hyperparameter optimization
      """
      n_estimators = n_estimators.round().astype(int)
      max_features = max_features.round().astype(int)

      rf_regressor = RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,random_state=17,n_jobs=-1)
      metric = cross_val_score(rf_regressor, X=X_train_2_opt, y=y_train_2_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'n_estimators': (20, 500),'max_features': (1,1) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_rf,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   | max_fe... | n_esti... |
-------------------------------------------------
| 1         | -23.74    | 1.0       | 274.7     |
| 2         | -24.05    | 1.0       | 52.59     |
| 3         | -23.74    | 1.0       | 335.0     |
| 4         | -23.73    | 1.0       | 296.3     |
| 5         | -23.75    | 1.0       | 191.8     |
| 6         | -24.11    | 1.0       | 48.82     |


KeyboardInterrupt: 

In [30]:
# n_estimators= , max_features= ->
rf_regressor = RandomForestRegressor(n_estimators=296,max_features=4)
rf_regressor.fit(X_train_2_opt,y_train_2_opt.values.ravel())
y_pred_2_opt = rf_regressor.predict(X_test_2_opt)
print(np.sqrt(mean_squared_error(y_test_2_opt, y_pred_2_opt)))
#custom 36.47
# 34.925

34.75418809127862


In [ ]:
def hyperparameter_function_lasso(alpha, max_iter, ):
      """ Function for hyperparameter optimization
      """
      max_iter = max_iter.round().astype(int)

      lasso_regressor = Lasso(alpha=alpha,max_iter=max_iter,random_state=17)
      metric = cross_val_score(lasso_regressor, X=X_train_2_opt, y=y_train_2_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'alpha': (0.001, 1),'max_iter': (100,10000) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_lasso,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   |   alpha   | max_iter  |
-------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.015e+07, tolerance: 5.460e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.090e+07, tolerance: 5.449e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.946e+06, tolerance: 5.448e

| 1         | -3.602e+0 | 0.2954    | 5.353e+03 |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.050e+04, tolerance: 5.456e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.998e+06, tolerance: 5.460e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.080e+07, tolerance: 5.449e

| 2         | -2.863e+0 | 0.1923    | 772.2     |


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.945e+06, tolerance: 5.460e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.810e+06, tolerance: 5.449e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.823e+06, tolerance: 5.448e

| 8         | -1.701e+0 | 0.05214   | 6.559e+03 |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.108e+07, tolerance: 5.460e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.185e+07, tolerance: 5.449e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.057e+07, tolerance: 5.448e

| 9         | -2.449e+0 | 0.5522    | 6.015e+03 |


KeyboardInterrupt: 

In [31]:
# alpha= , max_iter= ->
lasso_regressor = Lasso(alpha=0.9457,max_iter=694)
lasso_regressor.fit(X_train_2_opt,y_train_2_opt )
y_pred_2_opt = lasso_regressor.predict(X_test_2_opt)
print(np.sqrt(mean_squared_error(y_test_2_opt, y_pred_2_opt)))
#custom 1714
# 2762/714/86

86.99903836960912


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.582e+07, tolerance: 6.817e+03
  model = cd_fast.enet_coordinate_descent(


In [ ]:
def hyperparameter_function_xgboost(eta, gamma ,max_depth,reg_lambda,reg_alpha ):
      """ Function for hyperparameter optimization
      """
      max_depth = max_depth.round().astype(int)

      xgb_regressor = XGBRegressor(eta=eta,gamma=gamma,max_depth=max_depth,reg_lambda=reg_lambda, reg_alpha=reg_alpha)
      metric = cross_val_score(xgb_regressor, X=X_train_2_opt, y=y_train_2_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'eta': (0, 1),'gamma': (0,2),'max_depth':(1,10),'reg_lambda':(1,1),'reg_alpha':(0,0) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_xgboost,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   |    eta    |   gamma   | max_depth | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
| 1         | -23.25    | 0.2947    | 1.061     | 2.724     | 0.0       | 1.0       |
| 2         | -26.7     | 0.6563    | 1.275     | 6.18      | 0.0       | 1.0       |
| 3         | -30.14    | 0.9457    | 0.1201    | 8.776     | 0.0       | 1.0       |
| 4         | -26.5     | 0.6524    | 1.104     | 6.378     | 0.0       | 1.0       |
| 5         | -24.26    | 0.2977    | 1.123     | 4.564     | 0.0       | 1.0       |
| 6         | -24.4     | 0.1439    | 0.3018    | 1.497     | 0.0       | 1.0       |
| 7         | -23.33    | 0.1988    | 1.663     | 6.112     | 0.0       | 1.0       |
| 8         | -23.37    | 0.159     | 1.354     | 2.066     | 0.0       | 1.0       |
| 9         | -27.95    | 0.7973    | 0.136     | 9.647     | 0.0       | 1.0       |
| 10        | -23.9     | 0.7436    | 1.775     | 2.20

In [32]:
# eta=0.02803, gamma=0.8998, max_depth=6,lambda=1 , alpha=0 -> 22.92
xgb_regressor = XGBRegressor(eta=0.02803,gamma=0.8998,max_depth=6,reg_lambda=1, reg_alpha=0)
xgb_regressor.fit(X_train_2_opt,y_train_2_opt )
y_pred_2_opt = xgb_regressor.predict(X_test_2_opt)
print(np.sqrt(mean_squared_error(y_test_2_opt, y_pred_2_opt)))

#custom 35/37 - 34.11

34.113645722674306


In [37]:
## Laden des vierten Datensatzes
train_data_4_opt, test_data_4_opt,test_rul_data_4_opt = load_data(config_path=PATH_TO_CONFIG, dataset_num=4)
cleaned_train_4_opt, cleaned_test_4_opt = clean_data(train_data_4_opt, test_data_4_opt, method=None, ignore_columns=['UnitNumber', 'Cycle','Operation Setting 2','Operation Setting 3','Sensor Measure 2','Sensor Measure 3','Sensor Measure 4','Sensor Measure 8','Sensor Measure 9','Sensor Measure 11','Sensor Measure 15','Sensor Measure 17'], threshold_missing=0.1, threshold_corr=0.5)

# Train Val Split
cl_train_4_opt, cl_val_4_opt = train_val_split_by_group(df = cleaned_train_4_opt,group = "UnitNumber",test_size = 0.18,n_splits = 2,random_state = 7)

## RollingWindowParameter
min_ts_4_opt = 17
max_ts_4_opt = 18
feature_list_ds_4 = ["c3", "quantile", "mean", "root_mean_square", "median", "time_reversal_asymmetry_statistic", "absolute_maximum", "maximum", "minimum", "agg_autocorrelation", "autocorrelation" ]

#rwCreator = RollingWindowDatasetCreator(max_timeshift=max_ts_4_opt,min_timeshift=min_ts_4_opt,feature_extraction_mode= 'custom',feature_list=feature_list_ds_4)
rwCreator = RollingWindowDatasetCreator(max_timeshift=max_ts_4_opt,min_timeshift=min_ts_4_opt,feature_extraction_mode= 'minimal',feature_list=["median"])


X_train_4_opt, y_train_4_opt = rwCreator._process_data(cl_train_4_opt,'train')
X_val_4_opt, y_val_4_opt = rwCreator._process_data(cl_val_4_opt, 'train')
X_test_4_opt, y_test_4_opt = rwCreator._process_data(cleaned_test_4_opt, 'test', test_rul_data_4_opt)

y_train_4_opt = y_train_4_opt.clip(upper=125)

scaler = StandardScaler()
X_train_4_opt[2:] = scaler.fit_transform(X_train_4_opt[2:])
X_val_4_opt[2:] = scaler.fit_transform(X_val_4_opt[2:])
X_test_4_opt[2:] = scaler.fit_transform(X_test_4_opt[2:])

2024-06-01 10:27:02 [src.utils:60] [INFO] >>>> Loading data set 4...


INFO:src.utils:Loading data set 4...


2024-06-01 10:27:03 [src.utils:89] [INFO] >>>> Loaded raw data for dataset 4.


INFO:src.utils:Loaded raw data for dataset 4.


2024-06-01 10:27:03 [src.utils:90] [INFO] >>>> Train Data: (61249, 26)


...

INFO:src.rolling_window_creator:Calculating target for train data...


2024-06-01 10:33:07 [src.rolling_window_creator:128] [INFO] >>>> Creating rolling windows for test data...


INFO:src.rolling_window_creator:Creating rolling windows for test data...
Rolling: 100%|██████████| 486/486 [00:56<00:00,  8.53it/s]


2024-06-01 10:34:05 [src.rolling_window_creator:134] [INFO] >>>> Extracting features for test data...


INFO:src.rolling_window_creator:Extracting features for test data...
Feature Extraction: 100%|██████████| 2480/2480 [00:00<00:00, 2517.20it/s]


In [46]:
def hyperparameter_function_knn(neighbours):
      """ Function for hyperparameter optimization
      """
      neighbours = neighbours.round().astype(int)
      knn_regressor = KNeighborsRegressor(n_neighbors=neighbours)
      metric = cross_val_score(knn_regressor, X=X_train_4_opt, y=y_train_4_opt, cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'neighbours': (1, 750)}

optimizer = BayesianOptimization(
  f=hyperparameter_function_knn,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   | neighb... |
-------------------------------------
| 1         | -29.2     | 221.7     |
| 2         | -30.7     | 398.4     |
| 3         | -28.2     | 144.4     |
| 4         | -26.42    | 51.86     |
| 5         | -31.78    | 590.5     |
| 6         | -31.27    | 492.6     |
| 7         | -31.2     | 478.5     |
| 8         | -30.92    | 432.1     |
| 9         | -25.86    | 30.26     |
| 10        | -29.68    | 269.0     |
| 11        | -25.88    | 32.08     |
| 12        | -32.41    | 750.0     |
| 13        | -31.2     | 1.0       |
| 14        | -27.17    | 84.2      |
| 15        | -27.7     | 112.9     |
| 16        | -30.21    | 329.7     |
| 17        | -32.1     | 668.4     |
| 18        | -28.74    | 181.1     |
| 19        | -26.83    | 67.09     |
| 20        | -31.53    | 541.6     |
| 21        | -25.81    | 26.12     |
| 22        | -30.46    | 363.1     |
| 23        | -26.08    | 41.45     |
| 24        | -25.77    | 23.34     |
| 25        

In [49]:
# Neigbors = 15
knn_regressor = KNeighborsRegressor(n_neighbors=15)
knn_regressor.fit(X_train_4_opt,y_train_4_opt )
y_pred_4_opt = knn_regressor.predict(X_test_4_opt)
print(np.sqrt(mean_squared_error(y_test_4_opt, y_pred_4_opt)))

42.10044525939633


In [50]:
def hyperparameter_function_rf(n_estimators, max_features, ):
      """ Function for hyperparameter optimization
      """
      n_estimators = n_estimators.round().astype(int)
      max_features = max_features.round().astype(int)

      rf_regressor = RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,random_state=17,n_jobs=-1)
      metric = cross_val_score(rf_regressor, X=X_train_4_opt, y=y_train_4_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'n_estimators': (20, 500),'max_features': (1,1) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_rf,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   | max_fe... | n_esti... |
-------------------------------------------------
| 1         | -22.6     | 1.0       | 274.7     |
| 2         | -22.88    | 1.0       | 52.59     |
| 3         | -22.6     | 1.0       | 335.0     |
| 4         | -22.6     | 1.0       | 296.3     |
| 5         | -22.6     | 1.0       | 191.8     |
| 6         | -22.83    | 1.0       | 48.82     |
| 7         | -22.62    | 1.0       | 441.1     |
| 8         | -22.6     | 1.0       | 333.2     |
| 9         | -22.61    | 1.0       | 306.8     |
| 10        | -22.59    | 1.0       | 155.8     |
| 11        | -22.6     | 1.0       | 232.5     |
| 12        | -22.61    | 1.0       | 394.5     |
| 13        | -22.6     | 1.0       | 486.0     |
| 14        | -22.61    | 1.0       | 125.1     |
| 15        | -22.61    | 1.0       | 367.7     |
| 16        | -23.16    | 1.0       | 20.01     |
| 17        | -22.62    | 1.0       | 464.0     |
| 18        | -22.62    | 1.0       | 417.5     |


KeyboardInterrupt: 

In [75]:
# n_estimators= 264, max_features= ->
rf_regressor = RandomForestRegressor(n_estimators=264,max_features=1,random_state=17)
rf_regressor.fit(X_train_4_opt,y_train_4_opt.values.ravel())
y_pred_4_opt = rf_regressor.predict(X_test_4_opt)
print(np.sqrt(mean_squared_error(y_test_4_opt, y_pred_4_opt)))

39.53115656390862


In [52]:
def hyperparameter_function_lasso(alpha, max_iter, ):
      """ Function for hyperparameter optimization
      """
      max_iter = max_iter.round().astype(int)

      lasso_regressor = Lasso(alpha=alpha,max_iter=max_iter,random_state=17)
      metric = cross_val_score(lasso_regressor, X=X_train_4_opt, y=y_train_4_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'alpha': (0.001, 1),'max_iter': (100,10000) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_lasso,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   |   alpha   | max_iter  |
-------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.244e+07, tolerance: 6.235e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.296e+07, tolerance: 6.228e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.266e+07, tolerance: 6.243e

| 1         | -1.191e+0 | 0.2954    | 5.353e+03 |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.039e+04, tolerance: 6.243e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.169e+07, tolerance: 6.235e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.222e+07, tolerance: 6.228e

| 2         | -9.29e+03 | 0.1923    | 772.2     |


...

| 12        | -856.6    | 0.6492    | 688.0     |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.325e+07, tolerance: 6.235e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.416e+07, tolerance: 6.228e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.364e+07, tolerance: 6.243e

| 13        | -611.8    | 0.7394    | 8.793e+03 |


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.270e+07, tolerance: 6.235e+03
  model = cd_fast.enet_coordinate_descent(


KeyboardInterrupt: 

In [53]:
# alpha= 0.9457, max_iter= 694 -> 52.03
lasso_regressor = Lasso(alpha=0.9457,max_iter=694)
lasso_regressor.fit(X_train_4_opt,y_train_4_opt )
y_pred_4_opt = lasso_regressor.predict(X_test_4_opt)
print(np.sqrt(mean_squared_error(y_test_4_opt, y_pred_4_opt)))

50.39355716657052


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.989e+07, tolerance: 7.792e+03
  model = cd_fast.enet_coordinate_descent(


In [54]:
def hyperparameter_function_xgboost(eta, gamma ,max_depth,reg_lambda,reg_alpha ):
      """ Function for hyperparameter optimization
      """
      max_depth = max_depth.round().astype(int)

      xgb_regressor = XGBRegressor(eta=eta,gamma=gamma,max_depth=max_depth,reg_lambda=reg_lambda, reg_alpha=reg_alpha)
      metric = cross_val_score(xgb_regressor, X=X_train_4_opt, y=y_train_4_opt.values.ravel(), cv=5, scoring='neg_root_mean_squared_error')
      return metric.min()

# Bounded region of parameter space
pbounds = {'eta': (0, 1),'gamma': (0,2),'max_depth':(1,10),'reg_lambda':(1,1),'reg_alpha':(0,0) }

optimizer = BayesianOptimization(
  f=hyperparameter_function_xgboost,
  pbounds=pbounds,
  random_state=17,
  allow_duplicate_points= True
)

optimizer.maximize(
  init_points=10,
  n_iter=50,
)

|   iter    |  target   |    eta    |   gamma   | max_depth | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
| 1         | -23.01    | 0.2947    | 1.061     | 2.724     | 0.0       | 1.0       |
| 2         | -25.45    | 0.6563    | 1.275     | 6.18      | 0.0       | 1.0       |
| 3         | -27.83    | 0.9457    | 0.1201    | 8.776     | 0.0       | 1.0       |
| 4         | -25.7     | 0.6524    | 1.104     | 6.378     | 0.0       | 1.0       |
| 5         | -23.11    | 0.2977    | 1.123     | 4.564     | 0.0       | 1.0       |
| 6         | -23.56    | 0.1439    | 0.3018    | 1.497     | 0.0       | 1.0       |
| 7         | -23.07    | 0.1988    | 1.663     | 6.112     | 0.0       | 1.0       |
| 8         | -22.76    | 0.159     | 1.354     | 2.066     | 0.0       | 1.0       |
| 9         | -26.8     | 0.7973    | 0.136     | 9.647     | 0.0       | 1.0       |
| 10        | -23.26    | 0.7436    | 1.775     | 2.20

In [57]:
# eta= 0.1149 , gamma= 0.4352, max_depth=4 ,lambda=1 , alpha=0 ->
xgb_regressor = XGBRegressor(eta=0.1149,gamma=0.4352,max_depth=4,reg_lambda=1, reg_alpha=0)
xgb_regressor.fit(X_train_4_opt,y_train_4_opt )
y_pred_4_opt = xgb_regressor.predict(X_test_4_opt)
print(np.sqrt(mean_squared_error(y_test_4_opt, y_pred_4_opt)))

43.26899689673861
